In [162]:
import numpy as np
from itertools import combinations

In [569]:
class CyclicCode:
    def __init__(self, p, n, generator = None):
        self.p = p
        self.n = n
        if generator:
            assert max(generator) < p
            self.gen = np.array(generator)
        self.deg_gen = self.__degree(self.gen)
        assert self.deg_gen < self.n
        
    def __degree(self, poly):
        l = np.nonzero(poly)[0]
        if len(l):
            return np.max(np.nonzero(poly))
        return 0
    
    def __weight(self, w):
#         print(np.nonzero(w))
        return len(np.nonzero(w)[0])
    
    def __index2vec(self, index: tuple):
        index = np.array(index) - 1
        return np.array([0 if i not in index else 1 for i in range(self.n)])
    
    def __mul_by_c(self, poly, c):
        """Multiply by a constant"""
        return poly * c % self.p
    
    def __c_div(self, n, c):
        """Divide a constant by a constant"""
        for i in range(self.p):
            if i * n % self.p == c:
                return i
    
    def __pad(self, w):
        v = np.zeros(self.n)
        v[:len(w)] = w
        return v
    
    def __shift(self, w, j):
        v = np.zeros(self.n)
        v[:len(w)] = w
        return np.roll(v, j)
    
    def __poly_mul(self, u, v):
        return np.convolve(u, v) % self.p
    
    def __monomial(self, deg):
        return np.array([0 for i in range(deg)] + [1])
        
    def __convert_eff(self, poly):
        """Convert efficient to range 0 to p - 1"""
        for i in range(len(poly)):
            if poly[i] >= self.p:
                poly[i] %= self.p
            while poly[i] < 0:
                poly[i] = (poly[i] + self.p) % self.p
        return poly
                
    def remainder(self, u, g):
        """Euclidean division"""
        deg_u = self.__degree(u)
        u = self.__convert_eff(u[:deg_u + 1])
        deg_g = self.__degree(g)
        g = self.__convert_eff(g[:deg_g + 1])
        r =  deg_u - deg_g
        if r < 0 or deg_u == 0:
            return u
        else:
#             print(self.__poly_mul(self.__mul_by_c(self.gen, self.__c_div(u[deg_u], self.gen[self.deg_gen])), self.__monomial(r)), u)
            return self.remainder(u - self.__poly_mul(self.__mul_by_c(self.gen, self.__c_div(u[deg_u], g[deg_g])), self.__monomial(r)), g)
    
    def encoding(self, u):
        l = len(u)
        assert l - 1 == self.deg_gen
        v = np.zeros(self.n)
        v[:l] = u
        u = np.flip(v)
        r = self.remainder(u, self.gen)
        u[:len(r)] += r
        return self.__convert_eff(u)
    
    def meggitt_decoding(self, word, t):
        """
        For simplicity, this function is implemented for p = 2 only.
        t is the number of errors
        """
        er = []
        for i in range(t):
            for j in combinations(np.arange(self.n), i):
                er.append(self.__index2vec(j + (self.n, )))
        syn_table = [self.remainder(e, self.gen) for e in er]
        word = self.__convert_eff(word)
        syn = self.remainder(word, self.gen)
        if sum(syn) == 0:
            return word
        for i in range(self.n - 1):
            syn = np.array(syn)
            for j in range(len(syn_table)):
                if np.array_equal(syn, syn_table[j]):
                    word[:len(er[j])] -= er[j]
                    word = self.__convert_eff(word)
                    break
            word = np.roll(word, 1)
            syn = self.remainder(self.__poly_mul(syn, np.array([0, 1])), self.gen)
        word = np.roll(word, 1)
        return word
        
    def error_trapping_decoding(self, word, t):
        assert self.p == 2
        word = self.__pad(np.array(word))
        for j in range(self.n + 1):
            word = self.__shift(word, j)
            syn = self.remainder(word, self.gen)
#             print(syn, word)
            if self.__weight(syn) <= t and self.__degree(word) >= self.deg_gen:
#                 print(syn, self.__weight(syn))
                word[:len(syn)] -= syn
                word = self.__shift(word, -j)
                return self.__convert_eff(word)
        return self.__convert_eff(word)

In [512]:
a=CyclicCode(3, 4, [1,0,2])

In [513]:
a.remainder([1,1,2,2], [1,0,2])

array([0])

In [514]:
data = np.random.randint(0, 3, 3)
data

array([0, 2, 0])

In [515]:
a.encoding(data)

array([2., 0., 2., 0.])

Hamming code (7,4) with generator polynomial $1 + x + x^3$

In [484]:
hm = CyclicCode(2, 7, [1, 1, 0, 1])

In [361]:
hm.remainder([1, 0, 1, 0, 0, 0, 1], hm.gen)

array([0])

In [362]:
word = np.array([1, 0, 1, 0, 0, 0, 1])
def noise(word, num_e):
    data = word.copy()
    corrupted_pos = np.random.randint(0, high = len(data) - 1, size = num_e)
    data[corrupted_pos] ^= 1
    return data, corrupted_pos
word, e = noise(word, 1)
word, e

(array([1, 0, 1, 0, 1, 0, 1]), array([4]))

In [485]:
hm.meggitt_decoding(word, 1)

array([1, 0, 1, 0, 0, 0, 1])

Error trapping decode

In [570]:
bch = CyclicCode(2, 15, [1, 0, 0, 0, 1, 0, 1, 1, 1])

In [563]:
data = [1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]

In [572]:
def burst_noise(word, t):
    noise = (np.arange(t) + np.random.randint(len(word))) % len(word)
    return [word[i] if i not in noise else word[i] ^ 1 for i in range(len(word))]
err = burst_noise(data, 2)
err

[1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]

In [573]:
bch.error_trapping_decoding(err, 2)

array([1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.])

In [559]:
err = [0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1]